# Declare global variables

### Gomoku environment consulted from https://github.com/rockingdingo/gym-gomoku/blob/master/gym_gomoku/envs/gomoku.py
### DQN consulted from https://github.com/doublejtoh/GomokuAI/blob/master/dqn.py

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

from environment.ReplayMemory import ReplayMemory
from environment.GomokuEnvironment import GomokuEnvironment
import math
import os
import random

# set global variables
num_actions = 225
num_states = 225
hidden_units = 1000
mem_threshold = 30000
batch = 50
num_epoch = 100
epsilon_disRate = 0.999
min_epsilon = 0.1
gamma = 0.99
learning_rate = 0.2
winning_reward = 1

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


## Set the network

In [2]:
# Set the model
X = tf.placeholder(tf.float32, [None, num_states])
weight1 = tf.Variable(tf.truncated_normal([num_states, hidden_units], stddev = 1.0 / math.sqrt(float(num_states))))
bias1 = tf.Variable(tf.truncated_normal([hidden_units], stddev = 0.01))
input_layer = tf.nn.relu(tf.matmul(X, weight1) + bias1)

weight2 = tf.Variable(tf.truncated_normal([hidden_units, hidden_units], stddev = 1.0 / math.sqrt(float(hidden_units))))
bias2 = tf.Variable(tf.truncated_normal([hidden_units], stddev = 0.01))
hidden_layer = tf.nn.relu(tf.matmul(input_layer, weight2) + bias2)

weight3 = tf.Variable(tf.truncated_normal([hidden_units, hidden_units], stddev = 1.0 / math.sqrt(float(hidden_units))))
bias3 = tf.Variable(tf.truncated_normal([hidden_units], stddev = 0.01))
hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer, weight3) + bias3)

weight4 = tf.Variable(tf.truncated_normal([hidden_units, num_actions], stddev = 1.0 / math.sqrt(float(hidden_units))))
bias4 = tf.Variable(tf.truncated_normal([num_actions], stddev = 0.01))
output_layer = tf.matmul(hidden_layer2, weight4) + bias4

Y = tf.placeholder(tf.float32, [None, num_actions])
cost = tf.reduce_sum(tf.square(Y - output_layer)) / (2 * batch)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


empty = 0
BlackStone = 1
whiteStone = 2

## Function for training

In [ ]:
def playGame(env, memory, sess, saver, epsilon, iteration):

        # Repeat playing
        num_wins = 0

        for i in range(num_epoch):
                env.reset()

                err = 0
                gameOver = False
                currentPlayer = BlackStone
                
                while( gameOver != True ):
                        
                        # Act
                        
                        action = - 9999
                        
                        if( currentPlayer == BlackStone ):
                                currentState = env.getState()
                        else:
                                currentState = env.getStateInverse()

                        if( (float(random.randrange(0, 9999)) / 10000) <= epsilon ):
                                action = env.getActionRandom()
                        else:
                                action = env.getAction(sess, currentState)

                        if( epsilon > min_epsilon ):
                                epsilon = epsilon * epsilon_disRate
                        
                        next_state, reward, gameOver = env.makeMove(currentPlayer, action)

                        if( reward == 1 and currentPlayer == BlackStone ):
                                num_wins = num_wins + 1

                        
                        # Learning
                
                        memory.remember(currentState, action, reward, next_state, gameOver)

                        inputs, targets = memory.getBatch(output_layer, batch, num_actions, num_states, sess, X)
                        
                        _, loss = sess.run([optimizer, cost], feed_dict = {X: inputs, Y: targets})
                        err = err + loss
                        
                        if( currentPlayer == BlackStone ):
                                currentPlayer = whiteStone
                        else:
                                currentPlayer = BlackStone

                print("num_epoch " + str(iteration) + str(i) + ": err = " + str(err) + ": Win count = " + str(num_wins) +
                                " Win ratio = " + str(float(num_wins) / float(i + 1) * 100))

                print(targets)

                if( (i % 10 == 0) and (i != 0) ):
                        save_path = saver.save(sess, os.getcwd() + "/GomokuModel.ckpt")
                        print("Model saved in file: %s" % save_path)
        return float(num_wins) / float(i + 1) * 100

## Run and Train

In [ ]:
print("Training new model")

# Instantiate the environment
env = GomokuEnvironment(15)

# Instantiate replay memory
memory = ReplayMemory(15, mem_threshold, gamma)

# Initialize tensorflow
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Saver
saver = tf.train.Saver()

# Load model
if( os.path.isfile(os.getcwd() + "/GomokuModel.ckpt.index") == True ):
        saver.restore(sess, os.getcwd() + "/GomokuModel.ckpt")
        print('Saved model is loaded!')

# Playing the game
iteration = 0
winRateList = []
for x in range(9000000):
        winRate = playGame(env, memory, sess, saver, 1, iteration)
        winRateList.append(winRate)
        print(winRateList)
        iteration += 1

        df = pd.DataFrame(winRateList, columns=["colummn"])
        df.to_csv('results/list2.csv', index=False)


# close session
sess.close()

Training new model
num_epoch 00: err = 1.11605117551: Win count = 0 Win ratio = 0.0
[[ 0.7000466   0.06221122 -0.04591615 ...,  0.87519741  0.4803111
   0.50214678]
 [ 0.81718695  0.00440256  0.01672528 ...,  0.93738687  0.4196257
   0.7278595 ]
 [ 0.61284703 -0.02726706 -0.14187515 ...,  0.79849815  0.53512228
   0.63725358]
 ..., 
 [ 0.75119525  0.09283787 -0.07102896 ...,  0.86778218  0.44806683
   0.50240862]
 [ 0.85522115  0.12943372 -0.0938039  ...,  0.87159365  0.51436573
   0.46868688]
 [ 0.0836743  -0.0248617  -0.03026998 ...,  0.16317593  0.09069854
   0.11914407]]
